<a href="https://colab.research.google.com/github/EthanBerlant/Jedha_Final_Project/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import keras 

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing import image, sequence

from keras.losses import MeanAbsolutePercentageError

from keras.utils import np_utils

from keras.layers.recurrent import LSTM, GRU
from sklearn.preprocessing import StandardScaler
#from sklearn.cross_validation import train_test_split
#from keras.layers.core import Activation, TimeDistributedDense, RepeatVector
from keras.callbacks import EarlyStopping, ModelCheckpoint

from __future__ import print_function

!git clone 'https://github.com/EthanBerlant/Jedha_Final_Project.git'

In [13]:
output_dim = 128

In [ ]:
# embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1")
# embeddings = embed(["cat is on the mat", "dog is in the fog"])

In [18]:
def EMBED(output_dim): # "Token based text embedding trained on English Google News 200B corpus."
  return hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1", output_shape=[output_dim], input_shape=[], dtype=tf.string)

def LSTM(output_dim):
  return keras.layers.recurrent.LSTM(output_dim)

def GRU(output_dim):
  return keras.layers.recurrent.GRU(output_dim)

In [19]:
model = Sequential()

model.add(EMBED(output_dim))
model.add(GRU(output_dim))
model.add(LSTM(output_dim))

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='MeanAbsolutePercentageError', optimizer='adam')

model.summary()

ValueError: ignored

In [ ]:
print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          nb_epoch=4, validation_data=(X_test, y_test), show_accuracy=True)
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

In [34]:
!git clone 'https://github.com/EthanBerlant/Jedha_Final_Project.git'

Cloning into 'Jedha_Final_Project'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 126 (delta 61), reused 94 (delta 34), pack-reused 0
Receiving objects: 100% (126/126), 9.32 MiB | 9.92 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [123]:
df = pd.read_json('Jedha_Final_Project/bbctweets.json')
df = df.drop('Tweet Id', axis=1).copy()

In [124]:
import fnmatch
def filter(tweet):
  new_tweet = []
  for word in tweet.split():
    if word not in fnmatch.filter(tweet.split(), 'http*') and '@' not in word:
      new_tweet.append(word)
  return ' '.join(new_tweet)

df['Text'] = df['Text'].apply(filter)